In [ ]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from sqlalchemy import create_engine
from pathlib import Path
import os
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
!pip install keras-tuner
import keras_tuner as kt

!pip install pandas psycopg2-binary SQLAlchemy

spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 890.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,854 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/u

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Rainfall_df = pd.read_csv("/content/drive/MyDrive/Rainfall_Project_folder/Rainfall_OG.csv")
Humidity_df = pd.read_csv("/content/drive/MyDrive/Rainfall_Project_folder/Rainfall_YB2.csv")

In [ ]:
Merged_df = pd.merge(Rainfall_df, Humidity_df, on="Date")
Merged_df.head()

,Date,TempAvgºF,TempMinºF,TempMaxºF,RainTotal,WindSpeed,Pressure,Raining,HumidityMax,HumidityAvg,HumidityMin
0,2000-01-01,50.00,33.08,66.92,0.000000,2.920444,30.247579,False,100.0,73.6,40.0
1,2000-01-02,57.02,42.98,69.98,0.000000,7.580726,30.223955,False,86.0,73.2,56.0
2,2000-01-03,64.94,55.04,73.94,0.000000,10.314759,30.150130,False,100.0,81.9,51.0
3,2000-01-04,62.06,51.08,73.04,0.314961,15.907098,29.884360,True,100.0,84.2,59.0
4,2000-01-05,39.02,26.96,51.08,0.000000,5.840887,30.247579,False,92.0,57.5,37.0


In [ ]:
# Drop date column
Current_Merged_df = Merged_df.drop(columns=["Date", "RainTotal", "TempAvgºF", "HumidityMax", "HumidityMin"])
Current_Merged_df.tail()

,TempMinºF,TempMaxºF,WindSpeed,Pressure,Raining,HumidityAvg
8832,51.08,80.06,6.462258,30.043822,False,57.5
8833,71.96,98.96,7.394315,29.881407,False,85.3
8834,73.04,95.00,2.920444,29.710133,False,82.2
8835,71.96,93.02,3.231129,29.781005,False,74.9
8836,71.06,98.06,4.784557,29.881407,True,77.8


In [ ]:
Rain_df = Current_Merged_df[Current_Merged_df["Raining"] == True]
Rain_df

,TempMinºF,TempMaxºF,WindSpeed,Pressure,Raining,HumidityAvg
3,51.08,73.04,15.907098,29.884360,True,84.2
6,30.92,57.92,3.106855,30.330263,True,70.9
8,41.00,51.98,3.355403,30.105835,True,91.0
9,46.04,66.02,11.619638,29.775099,True,87.4
17,21.92,30.92,3.106855,30.144224,True,87.8
...,...,...,...,...,...,...
8823,66.02,80.06,5.840887,30.085164,True,80.5
8824,66.02,75.92,5.343791,30.076305,True,87.4
8825,66.02,80.96,8.077823,29.996574,True,84.2
8828,66.92,82.04,8.699194,29.716039,True,77.7


In [ ]:
# Split our preprocessed data into our features and target arrays

y = Current_Merged_df["Raining"].values
X = Current_Merged_df.drop(columns=["Raining"]).values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
trues = [x for x in y_test if x == False]
len(trues)

1668

In [ ]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    nn_model.add(tf.keras.layers.Input(shape = (5,)))

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
# Kerastuner

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 22s]
val_accuracy: 0.7547511458396912

Best val_accuracy So Far: 0.8565610647201538
Total elapsed time: 00h 10m 42s


In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 7,
 'num_layers': 4,
 'units_0': 9,
 'units_1': 7,
 'units_2': 1,
 'units_3': 9,
 'units_4': 1,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0042'}

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

70/70 - 0s - loss: 0.3910 - accuracy: 0.8566 - 380ms/epoch - 5ms/step
Loss: 0.39100411534309387, Accuracy: 0.8565610647201538


In [ ]:
best_model.save("rainfall_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
results = best_model.predict(X_test)
results

70/70 [==============================] - 0s 5ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [ ]:
tf.math.confusion_matrix(labels = y_test, predictions = results, num_classes = 2)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[   2, 1666],
       [   0,  542]], dtype=int32)>